## Version 2
### accuracy: 0.9750367
- only delete punctuation + jieba + remove stopwords
- Tokenizer
- TfidfVectorizer
- Ensemble model

### 改进方向
- 存在非中文content，需要处理
- lightgbm参数调优，加入catboost分类器
- 融合 bert-like transformer(prob < x)

In [1]:
import pandas as pd
import string
import re
import jieba
from tqdm import tqdm
tqdm.pandas()
from tokenizers import (
    models,
    normalizers,
    pre_tokenizers,
    trainers,
    Tokenizer,
)

from datasets import Dataset
from transformers import PreTrainedTokenizerFast
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier

In [2]:
train_dir = '/kaggle/input/xhc-dataset/train_jieba.csv'
test_dir = '/kaggle/input/xhc-dataset/test_jieba.csv'
df1 = pd.read_csv(train_dir)
df2 = pd.read_csv(test_dir)

In [3]:
train = df1[['id', 'content', 'topic']]
test = df2[['id', 'content', 'topic']]
test

,id,content,topic
0,181,2023年，习近平主席亲自擘画引领，中国特色大国外交扎实推进。在推动构建人类命运共同体的崇高...,1
1,1763,当地时间11月16日，习近平主席向亚太经合组织（APEC）工商领导人峰会发表书面演讲，强调我...,1
2,1015,亚太经合组织第三十次领导人非正式会议在美国旧金山莫斯科尼中心举行。国家主席习近平出席会议并发...,1
3,7104,习近平出席亚太经合组织领导人同东道主嘉宾非正式对话会暨工作午宴央视网2023年11月17日 ...,1
4,2427,中国国家主席习近平将于11月14日应美国总统拜登邀请赴美举行中美领导人会晤，同时应邀出席亚太...,1
...,...,...,...
3400,1044,从头到尾来讲一遍，咱们按照惯例，先谈谈缅甸北部的那场风波。这场风波的源头是藏匿在缅甸北部的一...,23
3401,9043,缅北全线激战，敏昂莱誓言反击，冲突可能扩大化，中缅边境乱不得这几天，缅甸内战的爆发，引起了外...,23
3402,5570,近日，缅甸内战再次卷土重来，引起了国际社会广泛的关注。然而，这次的冲突异常纷乱，各种消息传来...,23
3403,5523,缅北战火越烧越大，中方做两手准备，高层直接去了缅甸首都清酒半栩换种角度分析故事，带你了解更有...,23


In [14]:
def get_stopword():
    stopwords = []
    with open('/kaggle/input/xhc-dataset/baidu_stopwords.txt', 'r', encoding='utf-8') as f:
        for line in f.readlines():
            stopwords.append(line.strip())
   
    stopwords = list(set(stopwords))
    return stopwords

def wordopt_cn(text):
    # replace punctuation with space
    no_punct = re.sub(pattern, ' ', text)
    no_punct = re.sub(spaces_pattern, ' ', no_punct)
    return str(no_punct)

def remove_stopwords(words):
    return [word for word in words if word not in stopwords]

def process_content(content):
    content = wordopt_cn(content)
    content = re.sub(spaces_pattern, ' ', content)
    words = jieba.cut(content)
    words = remove_stopwords(words)
    content = ' '.join(words)
    content = re.sub(spaces_pattern, ' ', content)
    return content

# Precompile the regular expressions
punctuation = "！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏." + string.punctuation
pattern = re.compile('[%s]' % re.escape(punctuation))
spaces_pattern = re.compile(r'\s+')
stopwords = get_stopword()
stopwords.extend(["年","月","日"])

In [15]:
train['content'] = train['content'].progress_apply(process_content)
train.to_csv('train_jieba.csv', index=False)

100%|██████████| 6810/6810 [04:42<00:00, 24.13it/s]
/tmp/ipykernel_33/919418590.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['content'] = train['content'].progress_apply(process_content)


In [16]:
test['content'] = test['content'].progress_apply(process_content)
test.to_csv('test_jieba.csv', index=False)

100%|██████████| 3405/3405 [02:21<00:00, 24.04it/s]
/tmp/ipykernel_33/1735621898.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['content'] = test['content'].progress_apply(process_content)


In [26]:
test

,id,content,topic,pred,prob
0,181,2023 习近平 主席 亲自 擘画 引领 中国 特色 大国 外交 扎实 推进 推动 构建 人...,1,22,0.639504
1,1763,当地 时间 11 16 习近平 主席 亚太经合组织 APEC 工商 领导人 峰会 发表 书面...,1,1,0.489602
2,1015,亚太经合组织 第三十次 领导人 非正式 会议 美国 旧金山 莫斯科 尼 中心 国家 主席 习...,1,1,0.950451
3,7104,习近平 出席 亚太经合组织 领导人 东道主 嘉宾 非正式 对话会 暨 工作 午宴 央视网 2...,1,1,0.580280
4,2427,中国 国家 主席 习近平 11 14 日应 美国 总统 拜登 邀请 赴美 中 美 领导人 会...,1,1,0.750170
...,...,...,...,...,...
3400,1044,从头到尾 来讲 一遍 惯例 先 谈谈 缅甸 北部 那场 风波 这场 风波 源头 藏匿在 缅甸...,23,0,0.250714
3401,9043,缅北 全线 激战 敏昂 莱 誓言 反击 冲突 扩大化 中 缅 边境 乱 几天 缅甸 内战 爆...,23,23,0.330516
3402,5570,近日 缅甸 内战 再次 卷土重来 国际 社会 关注 这次 冲突 异常 纷乱 消息 传来 人 ...,23,23,0.221172
3403,5523,缅北 战火 越烧 越大 中方 做 两手 高层 去 缅甸 首都 清酒 半栩换种 角度 分析 故...,23,23,0.279709


In [27]:
LOWERCASE = False
VOCAB_SIZE = 100_000

In [28]:
raw_tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))
raw_tokenizer.normalizer = normalizers.Sequence([normalizers.NFC()] + [normalizers.Lowercase()] if LOWERCASE else [])
raw_tokenizer.pre_tokenizer = pre_tokenizers.WhitespaceSplit()

special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=special_tokens)

dataset = Dataset.from_pandas(test[['content']])


def train_corp_iter():
    for i in range(0, len(dataset), 1000):
        yield dataset[i: i + 1000]["content"]


raw_tokenizer.train_from_iterator(train_corp_iter(), trainer=trainer)
tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)
tokenized_texts_test = []

for text in tqdm(test['content'].tolist()):
    tokenized_texts_test.append(tokenizer.tokenize(text))

tokenized_texts_train = []

for text in tqdm(train['content'].tolist()):
    tokenized_texts_train.append(tokenizer.tokenize(text))

100%|██████████| 6810/6810 [00:19<00:00, 351.76it/s]


In [29]:
def dummy(text):
    return text
vectorizer = TfidfVectorizer(lowercase=False, sublinear_tf=True, analyzer='word',
                             tokenizer=dummy,
                             preprocessor=dummy,
                             token_pattern=None, strip_accents='unicode'
                             )

vectorizer.fit(tokenized_texts_test)

vocab = vectorizer.vocabulary_

vectorizer = TfidfVectorizer(lowercase=False, sublinear_tf=True, vocabulary=vocab,
                             analyzer='word',
                             tokenizer=dummy,
                             preprocessor=dummy,
                             token_pattern=None, strip_accents='unicode'
                             )

X_train = vectorizer.fit_transform(tokenized_texts_train)
print(len(vocab))

89394


In [30]:
clf = MultinomialNB(alpha=0.02)
sgd_model = SGDClassifier(max_iter=8000, tol=1e-4, loss="modified_huber")
p6 = {
    'n_iter': 1500, 
    'verbose': -1, 
    'objective': 'multiclass', 
    'metric': 'multi_logloss', 
    'learning_rate': 0.05073909898961407,
    'colsample_bytree': 0.726023996436955, 
    'colsample_bynode': 0.5803681307354022, 
    'lambda_l1': 8.562963348932286,
    'lambda_l2': 4.893256185259296, 
    'min_data_in_leaf': 115, 
    'max_depth': 23, 
    'max_bin': 898,
    'num_class': 24
}
lgb = LGBMClassifier(**p6)

# Creating the ensemble model
ensemble = VotingClassifier(estimators=[
    ('mnb', clf),
    ('sgd', sgd_model),
    ('lgb', lgb)],
    weights=[0.1, 0.45, 0.45],
    voting='soft',
    n_jobs=-1)

In [31]:
Y_train = train['topic'].values
ensemble.fit(X_train, Y_train)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

VotingClassifier(estimators=[('mnb', MultinomialNB(alpha=0.02)),
                             ('sgd',
                              SGDClassifier(loss='modified_huber',
                                            max_iter=8000, tol=0.0001)),
                             ('lgb',
                              LGBMClassifier(colsample_bynode=0.5803681307354022,
                                             colsample_bytree=0.726023996436955,
                                             lambda_l1=8.562963348932286,
                                             lambda_l2=4.893256185259296,
                                             learning_rate=0.05073909898961407,
                                             max_bin=898, max_depth=23,
                                             metric='multi_logloss',
                                             min_data_in_leaf=115, n_iter=1500,
                                             num_class=24,
                                             objective='multiclass',
                                             verbose=-1))],
                 n_jobs=-1, voting='soft', weights=[0.1, 0.45, 0.45])

In [32]:
tf_test = vectorizer.transform(tokenized_texts_test)
pred = ensemble.predict(tf_test)
prob = ensemble.predict_proba(tf_test)
test["pred"] = pred
test["prob"] = prob.max(axis=1)

In [33]:
different = test[test['topic'] != test['pred']]
different

,id,content,topic,pred,prob
0,181,2023 习近平 主席 亲自 擘画 引领 中国 特色 大国 外交 扎实 推进 推动 构建 人...,1,22,0.708688
7,2861,APEC 峰会 前 公使 提出 四点 核心 利益 中 美 会晤 看 美 表现 一带 一路 国...,1,4,0.531314
40,4395,2023 亚太经合 会 APEC 领袖 峰会 今 15 日于 美国 旧金山 登场 泰国 总理...,1,0,0.478081
43,3236,焦点访谈 把舵 中美关系 领航 亚太 合作 央视网 2023 11 19 22 01 加载 ...,1,4,0.809848
60,3670,亚太 国家 地区 领导人 13 抵达 美国 亚太经合组织 APEC 第三十次 领导人 非正式...,1,4,0.751292
...,...,...,...,...,...
3096,7668,2022 11 28 上午 9 时 神舟 十五号 载人 飞行 新闻 发布会 酒泉卫星发射中心...,21,19,0.951715
3099,598,2023 12 31 日晚 国家 主席 习近平 发表 二 〇 二 四年 新年贺词 2013 ...,21,19,0.495566
3127,5424,神舟 十七号 发射成功 续写 飞天 梦想 建设 航天 强国 北京 时间 10 26 11 时...,21,20,0.953893
3132,6470,神舟 十六号 出舱 多个 首次 神 十七 10 发射 航天员 杨利伟 7 国外 航天 执行 ...,21,20,0.739466


In [34]:
accuracy = accuracy_score(test['topic'], test['pred'])
precision = precision_score(test['topic'], test['pred'], average='weighted')
recall = recall_score(test['topic'], test['pred'], average='weighted')
f1 = f1_score(test['topic'], test['pred'], average='weighted')
print(f'Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1: {f1}')

Accuracy: 0.9750367107195301
Precision: 0.9751873711544411
Recall: 0.9750367107195301
F1: 0.9750247374021094
